In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder,normalize
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
import imblearn
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import xgboost
import inspect
from collections import defaultdict
# from tabpfn import TabPFNClassifier
from catboost import CatBoostClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier
from sklearn.utils import class_weight
import optuna
from functools import partial
import logging
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
import time
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from tqdm.notebook import tqdm
from datetime import datetime
from numpy.random import seed
import random  
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold as KF, GridSearchCV,StratifiedKFold
from sklearn.preprocessing import RobustScaler

from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score, log_loss
from sklearn.svm import SVC

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
seed(42) 
random.seed(42)

In [3]:
train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
sample = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')
greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')

In [4]:
# test2 = train.sample(frac=0.3,random_state=42)
# test = test2.copy(deep=True)
# # test.drop(['Id','Class'],inplace=True, axis=1)
# y_true = test2['Class']
# test2.drop(['Class'],inplace=True, axis=1)

In [5]:
def get_traindf(train,test):
    train = train.copy(deep=True) #pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv') 
    test = test.copy(deep=True)
    greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')
    
    rows=[102,194,203,292,356,360,372,457,458,586] 
    train['Correction'] = 0
    train.loc[rows,'Correction']=1 

    first_category = train.EJ.unique()[0]
    train.EJ = train.EJ.eq(first_category).astype('int')
    
    # Step 1: Identify columns with null values
    columns_with_nulls = train.columns[train.isnull().any()].tolist()

    # Step 2: Create new columns suffixed with "_missing"
    for col in columns_with_nulls:
        new_col_name = col + '_missing'
        train[new_col_name] = 0  # Initialize the new columns with 0

    # Step 3: Assign 1 to the new columns where the corresponding original column has a null value
    for col in columns_with_nulls:
        new_col_name = col + '_missing'
        train[new_col_name] = train[col].isnull().astype(int)

    
    times = greeks.Epsilon.copy()
    times[greeks.Epsilon != 'Unknown'] = greeks.Epsilon[greeks.Epsilon != 'Unknown'].map(lambda x: datetime.strptime(x,'%m/%d/%Y').toordinal())
    times[greeks.Epsilon == 'Unknown'] = np.nan
    
    bq_median = train['BQ'].median()
    el_median = train['EL'].median() 

    train_pred_and_time = pd.concat((train, times), axis=1)
    train_pred_and_time['BQ']=train_pred_and_time['BQ'].fillna(bq_median)
    train_pred_and_time['EL']=train_pred_and_time['EL'].fillna(el_median)
    
    train_pred_and_time = feature(train_pred_and_time)
    
    return train_pred_and_time

In [6]:
def feature(data):
    X=data.copy(deep=True)
    X['out_GL'] = 0
    X.loc[X['GL']<1,'out_GL'] = X.loc[X['GL']<1,'GL'].map(lambda x : x-X.loc[X['GL']<1,'GL'].mean())
    X.loc[X['GL']>1.5,'out_GL'] = X.loc[X['GL']>1.5,'GL'].map(lambda x : x-X.loc[X['GL']>1.5,'GL'].mean())
    X.out_GL = X.out_GL.astype('float')
    X['DA*CS'] = np.log(X.DA*2 / X.CS**0.5) 

    X['dupind1'] = X.loc[:, ['AR', 'AY', 'BZ', 'DF', 'DV']].duplicated(keep=False).astype(int)
    X['dupind2'] = 1-X['dupind1']
    
    X.drop(['AM','BD ', 'BR','BC','CL'],inplace=True, axis=1) 
    
     
    
    return X

In [7]:
def balanced_log_loss(y_true, y_pred):
    # y_true: correct labels 0, 1
    # y_pred: predicted probabilities of class=1
    # calculate the number of observations for each class
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    # calculate the weights for each class to balance classes
    w_0 = 1 / N_0
    w_1 = 1 / N_1
    # calculate the predicted probabilities for each class
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    # calculate the summed log loss for each class
    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0))
    log_loss_1 = -np.sum(y_true * np.log(p_1))
    # calculate the weighted summed logarithmic loss
    # (factgor of 2 included to give same result as LL with balanced input)
    balanced_log_loss = 2*(w_0 * log_loss_0 + w_1 * log_loss_1) / (w_0 + w_1)
    # return the average log loss
    return balanced_log_loss/(N_0+N_1)

In [8]:
xgb = xgboost.XGBClassifier(**{'alpha': 0.0001, 'colsample_bytree': 0.7, 'lambda': 0.01, 
                               'learning_rate': 0.2, 'max_depth': 20, 
                               'min_child_weight': 10, 'subsample': 0.8})

    

cb=CatBoostClassifier(**{'auto_class_weights': 'Balanced', 'colsample_bylevel': 0.5, 'depth': 4, 
                         'l2_leaf_reg': 3, 'learning_rate': 0.05, 'verbose': False}) 

lgbm = LGBMClassifier(**{           'verbose'           : -1,
                                    'class_weight' : 'balanced',    
                                   'boosting_type'     : 'gbdt', #'dart',  #gbdt
                                   'random_state'      : 42,
                                   'colsample_bytree'  : 0.4,
                                   'learning_rate'     : 0.10,
                                   'max_depth'         : 3,
                                   'min_child_samples' : 5,
                                   'n_estimators'      : 150,
                                   'num_leaves'        : 40,
                                   'reg_alpha'         : 0.0001,
                                   'reg_lambda'        : 0.65,
                                   'subsample'         : 0.65}) 

lr = LogisticRegression(C=0.1,penalty = 'l2' , multi_class='ovr', tol=0.0001, verbose=0,
                                                   solver = 'lbfgs',max_iter=5000)

  
svc=SVC(**{'probability':True,'C': 0.01, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': -1})


In [9]:
class Ensemble2():
    def __init__(self):
        self.imputer = SimpleImputer(missing_values=np.nan, strategy='median')
#         self.sc = StandardScaler() 
        self.rc = RobustScaler() 
        self.classifiers =[lgbm,cb,lr]         
    
    def fit(self,X,y): 
        for classifier in self.classifiers:
            if classifier!=self.classifiers[0]:
                X1 = self.rc.fit_transform(X)                
                classifier.fit(X1,y)
            else :
                classifier.fit(X, y)
     
    def predict_proba(self, x):
        probabilities = {}
        i=1
        for classifier in self.classifiers:
            if classifier!=self.classifiers[0]:
                x1 = self.rc.transform(x)
                prob = classifier.predict_proba(x1)
            else:
                prob = classifier.predict_proba(x)
            probabilities[i]=prob
            i+=1 
        
#         averaged_probabilities = probabilities[1]*0.159 + probabilities[2]*0.244
#         + probabilities[3]*0.4831 + probabilities[4]*0.1139 #+ probabilities[4]*0.1
        
        averaged_probabilities = probabilities[1]*0.4 + probabilities[2]*0.4
        + probabilities[3]*0.2 
        
#         probabilities = np.stack([classifier.predict_proba(x) for classifier in self.classifiers])
#         averaged_probabilities = np.mean(probabilities, axis=0)
        class_0_est_instances = averaged_probabilities[:, 0].sum()
        others_est_instances = averaged_probabilities[:, 1:].sum()
        # Weighted probabilities based on class imbalance
        new_probabilities = averaged_probabilities * np.array([[1/(class_0_est_instances if i==0 else others_est_instances) for i in range(averaged_probabilities.shape[1])]])
        return new_probabilities / np.sum(new_probabilities, axis=1, keepdims=1) 

In [10]:
class Ensemble3():
    def __init__(self):
        self.imputer = SimpleImputer(missing_values=np.nan, strategy='median')
        self.rc = RobustScaler()
#         self.classifiers =[xgb,lgbm,cb,lr] 
        self.classifiers =[lgbm,cb,lr] 
         
    
    def fit(self,X,y): 
        for classifier in self.classifiers:
            if classifier!=self.classifiers[0]:
                X1 = self.rc.fit_transform(X)                
                classifier.fit(X1,y)
            else :
                classifier.fit(X, y)
     
    def predict_proba(self, x):
        probabilities = {}
        i=1
        for classifier in self.classifiers:
            if classifier!=self.classifiers[0]:
                x1 = self.rc.transform(x)
                prob = classifier.predict_proba(x1)
            else:
                prob = classifier.predict_proba(x)
            probabilities[i]=prob
            i+=1 
        
        averaged_probabilities = probabilities[1]*0.4 + probabilities[2]*0.4
        + probabilities[3]*0.2 
    
        return averaged_probabilities

In [11]:
def ScoreMetric(ytrue, ypred):     
    nc = np.bincount(ytrue);
    return log_loss(ytrue, ypred, sample_weight = 1 / nc[ytrue], eps=1e-15)

In [12]:
# import optuna 

# def objective(trial,classifiers,x_val,y_val):
#     # Define the weights of the ensemble as trial parameters (uniform distribution between 0 and 1)
     
#     num_classifiers = len(classifiers)
#     weights = [trial.suggest_uniform(f'weight_{i}', 0, 1) for i in range(num_classifiers)]

# #     weights = [trial.suggest_float(f'weight_{i}', 0, 1/num_classifiers) for i in range(num_classifiers-1)]
# #     weights.append(1.0 - sum(weights))  # The last weight is computed to ensure they sum to 1

# #     Normalize the weights to sum up to 1
#     total_weight = sum(weights)
#     weights = [w / total_weight for w in weights]  
     

    
#     # Create the ensemble predictions using the weighted average of each classifier's predictions
#     ensemble_predictions = sum(weight * classifier.predict_proba(x_val) for weight, classifier in zip(weights, classifiers))

#     # Choose the final predicted class based on the maximum probability
# #     y_pred = np.argmax(ensemble_predictions, axis=1)

#     # Calculate the metric to be maximized (e.g., accuracy)
# #     metric = accuracy_score(y_test, y_pred)
#     metric = ScoreMetric(y_val, 1-ensemble_predictions[:,0])

#     return metric


In [13]:
n_splits=10
cv_innerz = KF(n_splits, shuffle=True, random_state=42)
def training1(data,greeks): 
    train_pred = data.copy(deep=True)
#     train_pred  = feature(train_pred )     
    
    y  = train_pred.Class
    x  = train_pred.drop(['Class', 'Id'],axis=1) 
    le = LabelEncoder()   
#     imputer = SimpleImputer(missing_values=np.nan, strategy='median')
     
    imputer = KNNImputer(n_neighbors=4) #, weights="distance")
#     sc=StandardScaler()
    
    outer_results = list()
    best_loss = np.inf
    split = 0
    splits = n_splits
    models = {}
    bestwt=[]
    bll = []
    for train_idx,val_idx in tqdm(cv_innerz.split(x), total = splits):
        split+=1
        x_train, x_val = x.iloc[train_idx],x.iloc[val_idx]
        y_train, y_val = greeks.Alpha.iloc[train_idx], y.iloc[val_idx]
        y_meta1 = le.fit_transform(y_train)  
        
        ros = RandomOverSampler(random_state=42)
        x_train1,y_train1 = ros.fit_resample(x_train, y_meta1) 
        
        x_train1 = imputer.fit_transform(x_train1)
        x_val = imputer.transform(x_val)
        
#         x_train1 = sc.fit_transform(x_train1)
#         x_val = sc.transform(x_val)
        
        model = Ensemble2() 

        model.fit(x_train1, y_train1) #,overwrite_warning=True)
        models[split]=model
        y_pred = model.predict_proba(x_val)
        
        probabilities = np.concatenate((y_pred[:,:1], np.sum(y_pred[:,1:], 1, keepdims=True)), axis=1)
        p0 = probabilities[:,0] 
        
        loss1 = balanced_log_loss(y_val,1-p0)
        loss = ScoreMetric(y_val,1-p0)
        print("balanced log loss ",loss1)
        print("ScoreMetric ",loss)
        
        bll.append(1/loss)

        if loss<best_loss:
            best_model = model
            best_loss = loss
            print('best_model_saved')             
            
        outer_results.append(loss)
        print('>val_loss=%.5f, split = %.1f' % (loss,split))
    print('LOSS: %.5f' % (np.mean(outer_results)))
    wtratios = bll/sum(bll)
    print("weights ",bll)
    print("weight ratios ",wtratios)
    
#     bestwts = np.stack(bestwt)
#     bestmean = np.mean(bestwts, axis=0)
#     print("mean weights are ",bestmean)

    return best_model, models, wtratios

train_pred_and_time = get_traindf(train,test)
m,models,wtratios = training1(train_pred_and_time,greeks)
    
#xgb - 0.30
#lgbm1 - 0.35
#cb - 0.30
#lr = 0.48
#svc1 - 0.86
#svc - 0.78    

#LOSS: 18060 vs 0.18347

#LOSS: 0.16929  LOSS: 0.18079

  0%|          | 0/10 [00:00<?, ?it/s]

balanced log loss  0.07548133586765199
ScoreMetric  0.11387372648165398
best_model_saved
>val_loss=0.11387, split = 1.0
balanced log loss  0.06851762246378984
ScoreMetric  0.12662583689942702
>val_loss=0.12663, split = 2.0
balanced log loss  0.03758688116357446
ScoreMetric  0.05123545077758164
best_model_saved
>val_loss=0.05124, split = 3.0
balanced log loss  0.09324233061161723
ScoreMetric  0.15972527578924098
>val_loss=0.15973, split = 4.0
balanced log loss  0.046122393257441885
ScoreMetric  0.11512628550753676
>val_loss=0.11513, split = 5.0
balanced log loss  0.06287644155050787
ScoreMetric  0.12667559817618043
>val_loss=0.12668, split = 6.0
balanced log loss  0.34627799842999524
ScoreMetric  0.4951981495405141
>val_loss=0.49520, split = 7.0
balanced log loss  0.06618496463356825
ScoreMetric  0.14520887582636055
>val_loss=0.14521, split = 8.0
balanced log loss  0.09342351928842703
ScoreMetric  0.14780140955452253
>val_loss=0.14780, split = 9.0
balanced log loss  0.1062768195551214
S

In [14]:
wtratios

array([0.11053986, 0.09940772, 0.24568117, 0.07880772, 0.1093372 ,
       0.09936867, 0.02541929, 0.08668606, 0.08516553, 0.0595868 ])

In [15]:
def get_testdf(train,test):
    train = train.copy(deep=True) #pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv') 
    test = test.copy(deep=True) #pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
    greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')
    
    test['Correction'] = 0 

    first_category = train.EJ.unique()[0]     
    test.EJ = test.EJ.eq(first_category).astype('int')    
    
    # Step 1: Identify columns with null values
    columns_with_nulls = train.columns[train.isnull().any()].tolist()

    # Step 2: Create new columns suffixed with "_missing"
    for col in columns_with_nulls:
        new_col_name = col + '_missing'
        test[new_col_name] = 0  # Initialize the new columns with 0

    # Step 3: Assign 1 to the new columns where the corresponding original column has a null value
    for col in columns_with_nulls:
        new_col_name = col + '_missing'
        test[new_col_name] = test[col].isnull().astype(int)

    from datetime import datetime
    times = greeks.Epsilon.copy()
    times[greeks.Epsilon != 'Unknown'] = greeks.Epsilon[greeks.Epsilon != 'Unknown'].map(lambda x: datetime.strptime(x,'%m/%d/%Y').toordinal())
    times[greeks.Epsilon == 'Unknown'] = np.nan
    
#     bq_median = train['BQ'].median()
#     el_median = train['EL'].median() 
    
    bq_median = test['BQ'].median()
    el_median = test['EL'].median() 
    test['BQ']=test['BQ'].fillna(bq_median)
    test['EL']=test['EL'].fillna(el_median)
    
    train_pred_and_time = pd.concat((train, times), axis=1)
    test['Epsilon']=train_pred_and_time.Epsilon.max() + 1
    test = feature(test)
    
    train1 = get_traindf(train,test)
    predictor_columns = [n for n in train1.columns if n != 'Class' and n != 'Id']
    test_predictors = test[predictor_columns] 
    return test_predictors

In [16]:
def get_traindf1(train,test):
    train = train.copy(deep=True) #pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv') 
    test = test.copy(deep=True)
    greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')
     
    rows=[102,194,203,292,356,360,372,457,458,586] 
    train['Correction'] = 0
    train.loc[rows,'Correction']=1 

    first_category = train.EJ.unique()[0]
    train.EJ = train.EJ.eq(first_category).astype('int')
    test.EJ = test.EJ.eq(first_category).astype('int')
    
    
    # Step 1: Identify columns with null values
    columns_with_nulls = train.columns[train.isnull().any()].tolist()

    # Step 2: Create new columns suffixed with "_missing"
    for col in columns_with_nulls:
        new_col_name = col + '_missing'
        train[new_col_name] = 0  # Initialize the new columns with 0

    # Step 3: Assign 1 to the new columns where the corresponding original column has a null value
    for col in columns_with_nulls:
        new_col_name = col + '_missing'
        train[new_col_name] = train[col].isnull().astype(int)

    from datetime import datetime
    times = greeks.Epsilon.copy()
    times[greeks.Epsilon != 'Unknown'] = greeks.Epsilon[greeks.Epsilon != 'Unknown'].map(lambda x: datetime.strptime(x,'%m/%d/%Y').toordinal())
    times[greeks.Epsilon == 'Unknown'] = np.nan
    
    bq_median = train['BQ'].median()
    el_median = train['EL'].median() 

    train_pred_and_time = pd.concat((train, times), axis=1)
    train_pred_and_time['BQ']=train_pred_and_time['BQ'].fillna(bq_median)
    train_pred_and_time['EL']=train_pred_and_time['EL'].fillna(el_median)
    
    train_pred_and_time = feature1(train_pred_and_time)
    
    return train_pred_and_time

def feature1(data):
    X=data.copy(deep=True)
    X['out_GL'] = 0
    X.loc[X['GL']<1,'out_GL'] = X.loc[X['GL']<1,'GL'].map(lambda x : x-X.loc[X['GL']<1,'GL'].mean())
    X.loc[X['GL']>1.5,'out_GL'] = X.loc[X['GL']>1.5,'GL'].map(lambda x : x-X.loc[X['GL']>1.5,'GL'].mean())
    X.out_GL = X.out_GL.astype('float')
    X['DA*CS'] = np.log(X.DA*2 / X.CS**0.5) 
    X['dupind1'] = X.loc[:, ['AR', 'AY', 'BZ', 'DF', 'DV']].duplicated(keep=False).astype(int)
    X['dupind2'] = 1-X['dupind1']    


#     X.drop(['AM','BD ', 'BR', 'CF', 'CS', 'CU', 'CW ', 'EL', 'EU', 'FC', 'FS', 'GB', 'GI'],inplace=True, axis=1)    
#     X.drop(['AM','BD ', 'BR', 'CF', 'CS', 'CU', 'CW ', 'EL'],inplace=True, axis=1)
    
    return X

In [17]:
def get_testdf1(train,test):
    train = train.copy(deep=True) #pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv') 
    test = test.copy(deep=True) #pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
    greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')
    
    test['Correction'] = 0 

    first_category = train.EJ.unique()[0]     
    test.EJ = test.EJ.eq(first_category).astype('int')    
    
    # Step 1: Identify columns with null values
    columns_with_nulls = train.columns[train.isnull().any()].tolist()

    # Step 2: Create new columns suffixed with "_missing"
    for col in columns_with_nulls:
        new_col_name = col + '_missing'
        test[new_col_name] = 0  # Initialize the new columns with 0

    # Step 3: Assign 1 to the new columns where the corresponding original column has a null value
    for col in columns_with_nulls:
        new_col_name = col + '_missing'
        test[new_col_name] = test[col].isnull().astype(int)

    from datetime import datetime
    times = greeks.Epsilon.copy()
    times[greeks.Epsilon != 'Unknown'] = greeks.Epsilon[greeks.Epsilon != 'Unknown'].map(lambda x: datetime.strptime(x,'%m/%d/%Y').toordinal())
    times[greeks.Epsilon == 'Unknown'] = np.nan
    
#     bq_median = train['BQ'].median()
#     el_median = train['EL'].median() 
    
    bq_median = test['BQ'].median()
    el_median = test['EL'].median() 
    test['BQ']=test['BQ'].fillna(bq_median)
    test['EL']=test['EL'].fillna(el_median)
    
    train_pred_and_time = pd.concat((train, times), axis=1)
    test['Epsilon']=train_pred_and_time.Epsilon.max() + 1
    test = feature1(test)
    
    train1 = get_traindf1(train,test)
    predictor_columns = [n for n in train1.columns if n != 'Class' and n != 'Id']
    test_predictors = test[predictor_columns] 
    return test_predictors

In [18]:
n_splits=10
cv_innerz = KF(n_splits, shuffle=True, random_state=42)
ypred_model1 =[]
def model1_predict(data,greeks,test_predictors,wtratios): 
    train_pred = data.copy(deep=True)
#     train_pred  = feature(train_pred )  
#     test_predictors = feature(test_predictors)
    
    y  = train_pred.Class
    x  = train_pred.drop(['Class', 'Id'],axis=1) 
    le = LabelEncoder()   
#     imputer = SimpleImputer(missing_values=np.nan, strategy='median')
     
    imputer = KNNImputer(n_neighbors=4) #, weights="distance")
    rc=RobustScaler()
    
    outer_results = list()
    best_loss = np.inf
    split = 0
    splits = n_splits
    models = {}
    for train_idx,val_idx in tqdm(cv_innerz.split(x), total = splits):
        split+=1
        x_train, _ = x.iloc[train_idx],x.iloc[val_idx]
        y_train, _ = greeks.Alpha.iloc[train_idx], y.iloc[val_idx]
        y_meta1 = le.fit_transform(y_train)  
        
        x_val = test_predictors.copy(deep=True)
        
        ros = RandomOverSampler(random_state=42)
        x_train1,y_train1 = ros.fit_resample(x_train, y_meta1) 
        
        x_train1 = imputer.fit_transform(x_train1)
        x_val = imputer.transform(x_val)
        
#         x_train1 = rc.fit_transform(x_train1)
#         x_val = rc.transform(x_val)
        
        model = Ensemble3() 
        
        model.fit(x_train1, y_train1) #,overwrite_warning=True)
        models[split]=model
        y_pred = model.predict_proba(x_val)         
        
        ypred_model1.append(y_pred*wtratios[split-1]) 

    ypred_model1_final = np.stack(ypred_model1)
#     ypred_stack_final = np.concatenate(ypred_stack,axis=0)
    ypred_model1_final = np.mean(ypred_model1_final,axis=0)
    
    class_0_est_instances = ypred_model1_final[:, 0].sum()
    others_est_instances = ypred_model1_final[:, 1:].sum()
    # Weighted probabilities based on class imbalance
    new_probabilities = ypred_model1_final * np.array([[1/(class_0_est_instances if i==0 else others_est_instances) for i in range(ypred_model1_final.shape[1])]])
    ypred_model1_final = new_probabilities / np.sum(new_probabilities, axis=1, keepdims=1) 
    return ypred_model1_final

In [19]:
%%time 
test_predictors = get_testdf(train,test)
train_data = get_traindf(train,test)

ypred_model1_final = model1_predict(train_data,greeks,test_predictors,wtratios)

  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 3min 42s, sys: 1min 16s, total: 4min 58s
Wall time: 1min 28s


In [20]:
np.round(ypred_model1_final,3)

array([[0.5  , 0.074, 0.254, 0.172],
       [0.5  , 0.074, 0.254, 0.172],
       [0.5  , 0.074, 0.254, 0.172],
       [0.5  , 0.074, 0.254, 0.172],
       [0.5  , 0.074, 0.254, 0.172]])

In [21]:
n_splits = 10
cv_innerz = KF(n_splits, shuffle=True, random_state=42)
ypred_stack = []
bll2=[]
def train_stack(data,greeks,test_predictors):     
    train_pred = data.copy(deep=True)    
#     train_pred.drop(['BC','CL'],inplace=True, axis=1)
#     test_predictors.drop(['BC','CL'],inplace=True, axis=1)
#     train_pred  = feature(train_pred )   
#     test_predictors = feature1(test_predictors)
     
    y  = train_pred.Class
    x  = train_pred.drop(['Class', 'Id'],axis=1) 
      
    
    imputer = SimpleImputer(missing_values=np.nan, strategy='median')     
#     imputer = KNNImputer(n_neighbors=4) #, weights="distance")
    
#     rc=RobustScaler()
    sc=StandardScaler()
    
    outer_results = list()
    best_loss = np.inf
    split = 0
    splits = n_splits
    models = {}
    for train_idx,val_idx in tqdm(cv_innerz.split(x), total = splits): 
        split+=1
        x_train, x_val = x.iloc[train_idx],x.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
#         y_meta1 = le.fit_transform(y_train)   
        
        ros = RandomOverSampler(random_state=42)
        x_train1,y_train1 = ros.fit_resample(x_train, y_train) 
        x_train1 = imputer.fit_transform(x_train1)
        x_train1 = sc.fit_transform(x_train1)
#         x_val = imputer.transform(test_predictors)
        x_val = imputer.transform(x_val)
        x_val = sc.transform(x_val) 
         
        estimators = [  ('lgbm',lgbm),('cb',cb),('lr',lr),('svc',svc)]
         
        model = StackingClassifier(estimators=estimators, cv=5,stack_method = 'predict_proba',
                                         final_estimator=LogisticRegression(),
                                         passthrough = False,verbose=0)
        
        model.fit(x_train1, y_train1) #,overwrite_warning=True)
        models[split]=model
        probabilities = model.predict_proba(x_val)    
        
        class_0_est_instances = probabilities[:, 0].sum()
        others_est_instances = probabilities[:, 1:].sum()
        # Weighted probabilities based on class imbalance
        new_probabilities = probabilities * np.array([[1/(class_0_est_instances if i==0 else others_est_instances) for i in range(probabilities.shape[1])]])
        probabilities = new_probabilities / np.sum(new_probabilities, axis=1, keepdims=1)  
        p0 = probabilities[:,0]  
        
        loss1 = balanced_log_loss(y_val,1-p0) 
        loss = ScoreMetric(y_val,1-p0)
        bll2.append(1/loss)
        
        print("balanced log loss ",loss1)
        print("ScoreMetric ",loss)

        if loss<best_loss:
            best_model = model
            best_loss = loss
            print('best_model_saved')             
            
        outer_results.append(loss)
        print('>val_loss=%.5f, split = %.1f' % (loss,split)) 
        
    print("ScoreMetric ",ScoreMetric(y_val,1-p0))
    print('LOSS: %.5f' % (np.mean(outer_results)))
    wtratios2 = bll2/sum(bll2)
    print("weights ",bll2)
    print("weight ratios ",wtratios2) 
    return models, wtratios2
 
test_predictors1 = get_testdf1(train,test)

models, ratios = train_stack(get_traindf1(train,test),greeks, test_predictors1)   

# LOSS: 0.17899

  0%|          | 0/10 [00:00<?, ?it/s]

balanced log loss  0.11458167404106732
ScoreMetric  0.17286183477781109
best_model_saved
>val_loss=0.17286, split = 1.0
balanced log loss  0.03796756165549451
ScoreMetric  0.07016697452101969
best_model_saved
>val_loss=0.07017, split = 2.0
balanced log loss  0.024930223040257625
ScoreMetric  0.03398289977544338
best_model_saved
>val_loss=0.03398, split = 3.0
balanced log loss  0.0761654792042297
ScoreMetric  0.13047241624824377
>val_loss=0.13047, split = 4.0
balanced log loss  0.08895029884224444
ScoreMetric  0.22202918749973233
>val_loss=0.22203, split = 5.0
balanced log loss  0.08237715623522243
ScoreMetric  0.1659632015556578
>val_loss=0.16596, split = 6.0
balanced log loss  0.3663955343148142
ScoreMetric  0.5239674233281794
>val_loss=0.52397, split = 7.0
balanced log loss  0.07414902329755615
ScoreMetric  0.16268190783620665
>val_loss=0.16268, split = 8.0
balanced log loss  0.08510935229929574
ScoreMetric  0.13464791662656442
>val_loss=0.13465, split = 9.0
balanced log loss  0.0657

In [22]:
ratios

array([0.0637167 , 0.15697108, 0.32410964, 0.08441773, 0.04960693,
       0.06636523, 0.02102074, 0.06770381, 0.0817999 , 0.08428824])

In [23]:
n_splits = 10
cv_innerz = KF(n_splits, shuffle=True, random_state=42)
ypred_stack = []
def train_stack(data,greeks,test_predictors,ratios):     
    train_pred = data.copy(deep=True)    
     
    y  = train_pred.Class
    x  = train_pred.drop(['Class', 'Id'],axis=1) 
    le = LabelEncoder()   
    
    imputer = SimpleImputer(missing_values=np.nan, strategy='median')     
#     imputer = KNNImputer(n_neighbors=4) #, weights="distance")
    
    sc=StandardScaler()
    
    outer_results = list()
    best_loss = np.inf
    split = 0
    splits = n_splits
    models = {}
    for train_idx,val_idx in tqdm(cv_innerz.split(x), total = splits): 
        split+=1
        x_train, x_val = x.iloc[train_idx],x.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
#         y_meta1 = le.fit_transform(y_train)   
        
        ros = RandomOverSampler(random_state=42)
        x_train1,y_train1 = ros.fit_resample(x_train, y_train) 
        x_train1 = imputer.fit_transform(x_train1)
        x_train1 = sc.fit_transform(x_train1)
        x_val = imputer.transform(test_predictors)
#         x_val = imputer.transform(x_val)
        x_val = sc.transform(x_val) 
         
        estimators = [ ('lgbm',lgbm),('cb',cb),('lr',lr),('svc',svc)]
         
        model = StackingClassifier(estimators=estimators, cv=5,stack_method = 'predict_proba',
                                         final_estimator=LogisticRegression(),
                                         passthrough = False,verbose=0)
        
        model.fit(x_train1, y_train1)  
        models[split]=model
        probabilities = model.predict_proba(x_val)    
        
        ypred_stack.append(probabilities * ratios[split-1])
         

    ypred_stack_final1 = np.stack(ypred_stack) 
    ypred_stack_final2 = np.mean(ypred_stack_final1,axis=0)
    
    class_0_est_instances = ypred_stack_final2[:, 0].sum()
    others_est_instances = ypred_stack_final2[:, 1:].sum()
    # Weighted probabilities based on class imbalance
    new_probabilities = ypred_stack_final2 * np.array([[1/(class_0_est_instances if i==0 else others_est_instances) for i in range(ypred_stack_final2.shape[1])]])
    ypred_stack_final = new_probabilities / np.sum(new_probabilities, axis=1, keepdims=1)  
    return ypred_stack_final
 
test_predictors1 = get_testdf(train,test)

ypred_stack_final = train_stack(get_traindf(train,test),greeks, test_predictors1,ratios)  


  0%|          | 0/10 [00:00<?, ?it/s]

In [24]:
ypred_stack_final 

array([[0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5]])

In [25]:
probabilities1 = np.concatenate((ypred_model1_final[:,:1], np.sum(ypred_model1_final[:,1:], 1, keepdims=True)), axis=1)
# probabilities2 = np.concatenate((ypred_model2_final[:,:1], np.sum(ypred_model2_final[:,1:], 1, keepdims=True)), axis=1)

# probabilities = 0.3*probabilities1 + 0.4*probabilities2 + 0.3 * ypred_stack_final

probabilities = 0.5*probabilities1 + 0.5 * ypred_stack_final

p0 = probabilities[:,:1]   

In [26]:
np.round(probabilities1,2)

array([[0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5]])

In [27]:
ypred_stack_final

array([[0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5]])

In [28]:
submission = pd.DataFrame(test["Id"], columns=["Id"])
submission["class_0"] = p0
submission["class_1"] = 1 - p0
submission.to_csv('submission.csv', index=False)

In [29]:
submission_df = pd.read_csv('submission.csv')
submission_df

,Id,class_0,class_1
0,00eed32682bb,0.5,0.5
1,010ebe33f668,0.5,0.5
2,02fa521e1838,0.5,0.5
3,040e15f562a2,0.5,0.5
4,046e85c7cc7f,0.5,0.5


In [30]:
# y_pred_tr = np.array(submission.loc[:,['class_0','class_1']])
# print(ScoreMetric(y_true,y_pred_tr[:,1]))

#0.018631567936169137
#0.023465416212992158
# @0.026254095804022815